Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

Rodzaj ruchów

In [6]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


Koszt

In [7]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

Pobór danych

In [8]:
# file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

file_path1 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/wera6/Downloads/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/wera6/Downloads/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/wera6/Downloads/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

Algorytm IHC

In [11]:
# Mapowanie typów sąsiedztwa na funkcje
neighborhood_functions = {
    "swap": swap_move,
    "two_opt": two_opt_move,
    "insertion": insertion_move
}

def hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations):
    current_route = initial_route
    current_distance = calculate_path_cost(distance_matrix, current_route)
    
    neighbor_func = neighborhood_functions[neighborhood_type]
    iterations_without_improvement = 0
    total_iterations = 0

    while iterations_without_improvement < max_iterations_without_improvement and total_iterations < max_hill_climbing_iterations:
        improved = False
        for neighbor in neighbor_func(current_route):
            new_distance = calculate_path_cost(distance_matrix, neighbor)
            if new_distance < current_distance:
                current_route = neighbor
                current_distance = new_distance
                improved = True
                iterations_without_improvement = 0
                break  # Skok do następnego sąsiedztwa

        if not improved:
            iterations_without_improvement += 1
        
        total_iterations += 1

    return current_route, current_distance

def iterated_hill_climbing(distance_matrix, num_starts=100, neighborhood_type="swap", time_limit=None, max_iterations_without_improvement=100, max_hill_climbing_iterations=1000):
    start_time = time.time()  # Zapisz czas rozpoczęcia

    n = len(distance_matrix)
    best_route = None
    best_distance = float('inf')
    best_time = float('inf')

    for start in range(num_starts):
        # Sprawdzenie limitu czasu
        if time_limit and (time.time() - start_time > time_limit):
            print(f"Przekroczono limit czasu ({time_limit} s).")
            break

        # Losowe rozwiązanie początkowe
        initial_route = list(range(n))
        random.shuffle(initial_route)

        # Wspinaczka
        hill_climbing_start_time = time.time()
        current_route, current_distance = hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations)
        hill_climbing_time = time.time() - hill_climbing_start_time

        # Aktualizacja najlepszego rozwiązania
        if current_distance < best_distance:
            best_route = current_route
            best_distance = current_distance
            best_time = hill_climbing_time

    return best_route, best_distance, best_time

Generowanie rozwiązań

In [12]:
# Metoda SWAP
swap_p_1, swap_c_1, swap_t_1= iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "swap", time_limit=1000)
swap_p_2, swap_c_2, swap_t_2= iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "swap", time_limit=1000)
swap_p_3, swap_c_3, swap_t_3= iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "swap", time_limit=1000)

Przekroczono limit czasu (1000 s).


In [13]:
# Metoda TWO OPT
opt_p_1, opt_c_1, opt_t_1= iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)
opt_p_2, opt_c_2, opt_t_2= iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)
opt_p_3, opt_c_3, opt_t_3= iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).


In [ ]:
# Metoda INSERTION
ins_p_1, ins_c_1, ins_t_1= iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "insertion", time_limit=1000)
ins_p_2, ins_c_2, ins_t_2= iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "insertion", time_limit=1000)
ins_p_3, ins_c_3, ins_t_3= iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "insertion", time_limit=1000)

Układ w DF

In [ ]:
W4 =  {
    "Długość ścieżki": [swap_c_1, opt_c_1, ins_c_1],
    "Ścieżka": [swap_p_1, opt_p_1, ins_p_1],
    "Czas": [swap_t_1, opt_t_1, ins_t_1]
}

i1 = pd.DataFrame(data = W4)
i1.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W5 =  {
    "Długość ścieżki": [swap_c_2, opt_c_2, ins_c_2],
    "Ścieżka": [swap_p_2, opt_p_2, ins_p_2],
    "Czas": [swap_t_2, opt_t_2, ins_t_2]
}

i2 = pd.DataFrame(data = W5)
i2.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W6 =  {
    "Długość ścieżki": [swap_c_3, opt_c_3, ins_c_3],
    "Ścieżka": [swap_p_3, opt_p_3, ins_p_3],
    "Czas": [swap_t_3, opt_t_3, ins_t_3]
}

i3 = pd.DataFrame(data = W6)
i3.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

Badanie wpływu parametru num_starts

In [ ]:
# Stałe parametry
time_limit = 1000

# DANE NR 1 
results = []
for num_starts in range(1, 200,  20):
    best_path, best_cost, best_time = iterated_hill_climbing(data1, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka" : best_path, "CZAS": best_time})


itterration1 = pd.DataFrame(results)



# DANE NR 2

results = []
for num_starts in range(1, 200,  20):
    best_path, best_cost, best_time = iterated_hill_climbing(data2, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK_2": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3

results = []
for num_starts in range(1, 200,  20):
    best_path, best_cost, best_time = iterated_hill_climbing(data3, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK_3": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

In [ ]:
merged_itt = itterration1.merge(itterration2, on="PARAMETR").merge(itterration3, on="PARAMETR")

Zapis do pliku

In [ ]:
resu = {
    "Porównanie_metod_1": i1,
    "Porównanie_metod_2": i2,
    "Porównanie_metod_3": i3,
    "max_iterations": merged_itt   
}

# Ścieżka do pliku Excel
file_name = "IHC.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")